In [4]:
import pandas as pd
import numpy as np

In [11]:
import os
notebook_dir = os.path.abspath("")
base_dir = os.path.dirname(notebook_dir)
experiment_data_folder = os.path.join(base_dir,"datasets", "NLP4CALL_2025_experiment","experiments_data","nlp4call2025_article_experiments")
# efcamdat_100k_with_text_and_measures.csv
efcamdat_100k_fp = os.path.join(experiment_data_folder, "efcamdat_100k_with_text_and_measures.csv")

## Expected folder structure
## 
```
./datasets
└── NLP4CALL_2025_experiment
    ├── experiments_data
    │   └── nlp4call2025_article_experiments
    │       ├── celva_1742_with_text_and_measures.csv
    │       ├── data_for_cefr_model_3efcamdat_to_1celva_with_ids_and_texts.csv
    │       ├── efcamdat_100k_with_text_and_measures.csv
    │       ├── efcamdat_test_with_id.csv
    │       └── efcamdat_train_with_id.csv
    └── experiments_data.zip
./notebooks
└── andrews-CEFR-benchmark-eval.ipynb
```

In [14]:
andrews100kdf = pd.read_csv(efcamdat_100k_fp,index_col=0)

In [15]:
andrews100kdf.head()


,writing_id,cefr_level,measures.collocations.text_level.ratio_num_token,measures.collocations.text_level.ttr,measures.counts.acl,measures.counts.acl_ratio,measures.counts.acl:relcl,measures.counts.acl:relcl_ratio,measures.counts.ADJ,measures.counts.ADJ_ratio,...,measures.taassc.L2SCA.CT_T,measures.taassc.L2SCA.DC_C,measures.taassc.L2SCA.DC_T,measures.taassc.L2SCA.MLC,measures.taassc.L2SCA.MLS,measures.taassc.L2SCA.MLT,measures.taassc.L2SCA.T_S,measures.taassc.L2SCA.VP_T,text,l1
0,115499,B1,0.108696,1.0,2,0.021739,1,0.010870,3,0.032609,...,0.125000,0.0,1.000000,0.0,11.500000,11.500000,1.000000,1.500000,grandmas home remedies and recipes. Do you hav...,German
1,1081381,A1,0.111111,1.0,0,0.000000,0,0.000000,6,0.166667,...,0.166667,0.0,0.666667,0.0,16.000000,5.333333,3.000000,1.000000,My friend is very nice.She comes from Italy.Sh...,Italian
2,452770,B1,0.176000,1.0,2,0.016000,1,0.008000,5,0.040000,...,0.111111,0.0,1.333333,0.0,125.000000,13.888889,9.000000,1.777778,"First, I will study a lot and finish my degree...",Portuguese
3,412035,A1,0.050000,1.0,0,0.000000,0,0.000000,2,0.050000,...,0.000000,0.0,0.750000,0.0,6.666667,10.000000,0.666667,1.500000,"Hy, my name's Andr. I have thirty one years ol...",Portuguese
4,132380,B1,0.061224,1.0,0,0.000000,1,0.010204,6,0.061224,...,0.222222,0.0,0.888889,0.0,8.636364,10.555556,0.818182,1.777778,bello! I glad to congratulate you with the bes...,Russian


In [46]:
cefr_levels =["A1","A2","B1","B2","C1","C2"]
sample_size = 10000
dummy_test_set = andrews100kdf.sample(sample_size).reset_index()
hard_golden_labels=dummy_test_set["cefr_level"].to_dict()
print(hard_golden_labels)

{0: 'A1', 1: 'B2', 2: 'A2', 3: 'B2', 4: 'A1', 5: 'A1', 6: 'A2', 7: 'B1', 8: 'B1', 9: 'A1', 10: 'A1', 11: 'A2', 12: 'A2', 13: 'A2', 14: 'A2', 15: 'A1', 16: 'A2', 17: 'B1', 18: 'B2', 19: 'B1', 20: 'A1', 21: 'B1', 22: 'A1', 23: 'B1', 24: 'A1', 25: 'A1', 26: 'A2', 27: 'A1', 28: 'A1', 29: 'A2', 30: 'A2', 31: 'B1', 32: 'A2', 33: 'A1', 34: 'A1', 35: 'A2', 36: 'A1', 37: 'A1', 38: 'A1', 39: 'A2', 40: 'A1', 41: 'A2', 42: 'A2', 43: 'A1', 44: 'A2', 45: 'A2', 46: 'A1', 47: 'A1', 48: 'B1', 49: 'A1', 50: 'A1', 51: 'A2', 52: 'B1', 53: 'A1', 54: 'A1', 55: 'A2', 56: 'B1', 57: 'A1', 58: 'A2', 59: 'B1', 60: 'A1', 61: 'B1', 62: 'A2', 63: 'B1', 64: 'B2', 65: 'A2', 66: 'A1', 67: 'B1', 68: 'B1', 69: 'A2', 70: 'B1', 71: 'A1', 72: 'A1', 73: 'A1', 74: 'A2', 75: 'A1', 76: 'A2', 77: 'A1', 78: 'A2', 79: 'B1', 80: 'A1', 81: 'A1', 82: 'A2', 83: 'A2', 84: 'A2', 85: 'A2', 86: 'A1', 87: 'A1', 88: 'A1', 89: 'B1', 90: 'A1', 91: 'A2', 92: 'A1', 93: 'B2', 94: 'A1', 95: 'B1', 96: 'A1', 97: 'A2', 98: 'A1', 99: 'B2', 100: 'B1'

In [47]:
random_probas = np.random.dirichlet(np.ones(5),size=sample_size)
'''predictions = {
    {
     "A1": 0.31,
     "A2": 0.54,
     "B1": 0.4,
     "B2": 0.74,
     "C1": 0.74,
    }
}'''
soft_predictions = {
    id_: {class_:proba for class_, proba in zip(cefr_levels, cefr_vector)}
    for id_, cefr_vector in zip(range(sample_size),random_probas.tolist())
}
idx_to_class_ = lambda v: {
         0: "A1",
         1: "A2",
         2: "B1",
         3: "B2",
         4: "C1",
}.get(v, None)
hard_predictions = {k:idx_to_class_(v) for k,v in zip(range(sample_size), np.argmax(random_probas,axis=1).tolist())}

In [48]:
from sklearn.metrics import classification_report

class hard_predictions_eval:
    def __init__(self, hard_predictions_dict : dict, hard_gold_labels_dict : dict):
        self.hard_predictions_dict = hard_predictions_dict
        self.hard_gold_labels_dict = hard_gold_labels_dict
        self.y_pred = [v for v in self.hard_predictions_dict.values()], 
        self.y_true = [v for v in self.hard_gold_labels_dict.values()]
        print(self.y_pred, self.y_true)
    def accuracy(self):
        pass
    def precision(self):
        pass
    def recall(self):
        pass
    def report(self):
        # assuming dicts are aligned with all same ids
        self.report = classification_report(
            [v for v in self.hard_predictions_dict.values()], 
            [v for v in self.hard_gold_labels_dict.values()]
        )
        print(self.report)
        

In [49]:
def soft_predictions_eval(predictions, gold_labels):
    pass

In [50]:
hard_eval = hard_predictions_eval(hard_predictions,hard_golden_labels)

(['A1', 'B2', 'A1', 'A1', 'B1', 'B1', 'B2', 'C1', 'A2', 'C1', 'B1', 'C1', 'C1', 'A2', 'C1', 'A2', 'C1', 'A2', 'A1', 'B1', 'C1', 'B1', 'C1', 'A1', 'B1', 'A2', 'B2', 'C1', 'B1', 'B1', 'B2', 'B1', 'B2', 'B2', 'A1', 'A2', 'B1', 'B2', 'A2', 'B2', 'C1', 'A1', 'C1', 'A2', 'B1', 'A1', 'B1', 'A1', 'A1', 'A1', 'B1', 'B2', 'A2', 'B1', 'A1', 'B2', 'B2', 'B1', 'B2', 'B2', 'A2', 'C1', 'B2', 'B2', 'A1', 'B2', 'A1', 'C1', 'B2', 'B2', 'A1', 'C1', 'C1', 'C1', 'A1', 'A1', 'A2', 'B2', 'C1', 'A2', 'B2', 'B1', 'B1', 'A1', 'A1', 'B2', 'B2', 'C1', 'A2', 'A1', 'B2', 'A2', 'A1', 'C1', 'B1', 'C1', 'C1', 'B1', 'C1', 'B2', 'A2', 'B2', 'A2', 'B1', 'B2', 'C1', 'C1', 'B1', 'C1', 'C1', 'A1', 'C1', 'B2', 'A1', 'A1', 'A1', 'B1', 'C1', 'A2', 'A2', 'B2', 'A1', 'A1', 'A2', 'A2', 'C1', 'C1', 'B1', 'B1', 'B2', 'B1', 'A2', 'A2', 'B1', 'C1', 'C1', 'B1', 'B2', 'A1', 'C1', 'B2', 'A1', 'A1', 'B1', 'A1', 'B2', 'A2', 'B1', 'B1', 'A2', 'B2', 'A2', 'A2', 'B1', 'A1', 'B1', 'C1', 'C1', 'C1', 'A2', 'B2', 'B1', 'B2', 'B2', 'A2', 'C1', 'A

In [51]:
hard_eval.report()

              precision    recall  f1-score   support

          A1       0.20      0.47      0.28      1969
          A2       0.19      0.29      0.23      2010
          B1       0.21      0.16      0.18      2097
          B2       0.19      0.06      0.09      1985
          C1       0.15      0.01      0.02      1939

    accuracy                           0.20     10000
   macro avg       0.19      0.20      0.16     10000
weighted avg       0.19      0.20      0.16     10000



### What is the distribution of CEFR level ?


In [45]:
andrews100kdf["cefr_level"].value_counts()/len(andrews100kdf["cefr_level"])

cefr_level
A1    0.47215
A2    0.29701
B1    0.16172
B2    0.05497
C1    0.01415
Name: count, dtype: float64